In [19]:
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.metrics import silhouette_score
import pandas as pd

milk = pd.read_csv(r"..\Cases\Recency Frequency Monetary\rfm_data_customer.csv", index_col=0)
milk = milk.drop('most_recent_visit', axis=1)
milk.info()



<class 'pandas.core.frame.DataFrame'>
Index: 39999 entries, 22086 to 19184
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   revenue           39999 non-null  int64
 1   number_of_orders  39999 non-null  int64
 2   recency_days      39999 non-null  int64
dtypes: int64(3)
memory usage: 1.2 MB


In [20]:
from sklearn.preprocessing import StandardScaler
# Create scaler: scaler
scaler = StandardScaler().set_output(transform='pandas')
milkscaled=scaler.fit_transform(milk)

clust_DB = DBSCAN(eps=1, min_samples=2)
clust_DB.fit(milkscaled)
print(clust_DB.labels_)

clust_milk = milk.copy()
clust_milk["Clust"] = clust_DB.labels_
clust_milk.sort_values(by='Clust')

clust_milk.groupby('Clust').mean()
clust_milk.sort_values('Clust')



[0 0 0 ... 0 0 0]


,revenue,number_of_orders,recency_days,Clust
customer_id,,,,
22086,777,9,232,0
33150,1178,14,85,0
9942,1197,12,370,0
15845,1108,9,127,0
3918,1535,15,132,0
...,...,...,...,...
12225,907,13,89,0
29210,1174,12,65,0
23402,687,10,373,0


In [22]:

milkscaled['Clust'] = clust_DB.labels_
milk_scl_inliers = milkscaled[milkscaled['Clust']!=-1]
# milk_scl_inliers.iloc[:,-1]
# print( silhouette_score(milk_scl_inliers.iloc[:,:-1],milk_scl_inliers.iloc[:,-1]) )


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [24]:

eps_range = [0.6,1,1.5,2]
mp_range = [20,50,100]
cnt = 0
a =[]
for i in eps_range:
    for j in mp_range:
        clust_DB = DBSCAN(eps=i, min_samples=j)
        clust_DB.fit(milkscaled.iloc[:,:5])
        if len(set(clust_DB.labels_)) > 2:
            cnt = cnt + 1
            milkscaled['Clust'] = clust_DB.labels_
            milk_scl_inliers = milkscaled[milkscaled['Clust']!=-1]
            sil_sc = silhouette_score(milk_scl_inliers.iloc[:,:-1],
                             milk_scl_inliers.iloc[:,-1])
            a.append([cnt,i,j,sil_sc])
            print(i,j,sil_sc)
    
a = np.array(a)
pa = pd.DataFrame(a,columns=['Sr','eps','min_pt','sil'])
print("Best Paramters:")
pa[pa['sil'] == pa['sil'].max()]

### Labels with best parameters
clust_DB = DBSCAN(eps=0.4, min_samples=2)
clust_DB.fit(milkscaled.iloc[:,:5])
print(clust_DB.labels_)

clust_milk = milk.copy()
clust_milk["Clust"] = clust_DB.labels_
clust_milk.sort_values(by='Clust')


clust_milk.groupby('Clust').mean()
clust_milk.sort_values('Clust')


print(clust_milk['Clust'].value_counts())

0.6 20 -0.025563719927770283
0.6 50 -0.002585616649219029
0.6 100 -0.0005640289583192534
1 20 -0.01875849551129519
1 50 -0.00771233545313779
1 100 -0.007773296400534713
1.5 20 0.01735722368247803
1.5 50 0.0010936006603708764
1.5 100 -0.06841080299908431
2 20 0.0813041430636347
2 50 0.07553719426957527
2 100 0.13671897355724344
Best Paramters:
[0 0 0 ... 0 0 0]
Clust
 0     38881
 1       829
 3       209
-1        27
 4        19
 5        11
 2         6
 8         6
 10        3
 6         2
 7         2
 9         2
 11        2
Name: count, dtype: int64
